In [ ]:
import statsmodels.api as sm
import pandas as pd
import datetime as dt
import yfinance as yf
import pandas_datareader.data as reader

In [ ]:
ticker = "FDGRX"
days = 365
years = 5
end = dt.date(2020,6,30)
start = end - dt.timedelta(days=days*years)


In [ ]:
data = yf.Ticker(ticker).history(start=start, end=end)[["Close"]]
factors = reader.DataReader("F-F_Research_Data_Factors", "famafrench", start, end)[0]#[1:]

In [ ]:
data["Returns"] = data["Close"].pct_change()
data_mtl = data["Returns"].resample("M").agg(lambda x: (x+1).prod()-1)
print(data_mtl.index[0], factors.index[0])
data_mtl.index = factors.index
factors /= 100
fund_df = pd.merge(data_mtl, factors, on="Date")
fund_df["Returns"] = fund_df["Returns"] - fund_df["RF"]

In [ ]:
y = fund_df["Returns"]
x_data = fund_df[fund_df.columns.difference(["Returns", "RF"])]
x = sm.add_constant(x_data)

In [ ]:
model = sm.OLS(y, x)
results = model.fit()
results.summary()